In [1]:
# 导入 torch 库，这是 PyTorch 的核心库，我们将用它来构建神经网络的所有部分。
import torch
# 导入 torch.nn 模块，它包含了所有构建神经网络所需的类和函数，比如线性层、Dropout层等。
# 我们给它一个别名 nn，这是 PyTorch 的一个常用约定。
import torch.nn as nn
# 导入 math 库，这是一个 Python 的标准数学库，我们将用它来进行一些数学计算，比如 sin, cos, log 等。
import math

# 这行代码是用来设置 PyTorch 在哪个设备上运行。
# torch.cuda.is_available() 会检查你的环境是否支持 NVIDIA 的 CUDA，也就是是否能用 GPU。
# 如果可以用 GPU，device 就被设置为 "cuda"；否则，就用 "cpu"。
# 在我们刚刚设置好的 Colab 环境中，这里会选择 "cuda"。
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 打印一下我们当前使用的设备，确认一下 GPU 是否设置成功。
print(f"当前使用的设备是: {device}")


当前使用的设备是: cuda


In [2]:
# 【修正后的 PositionalEncoding 类】
# 定义一个名为 PositionalEncoding 的类。
# 它继承自 nn.Module，这是 PyTorch 中所有神经网络模块的基类。
# 继承它意味着我们的 PositionalEncoding 类会自动获得很多有用的功能（比如参数管理）。
class PositionalEncoding(nn.Module):

    # 这是类的构造函数（initializer），当我们创建一个 PositionalEncoding 的实例时，这个函数会被自动调用。
    # 它接收三个参数：
    # d_model: 词嵌入的维度（比如 512）。位置编码的维度需要和词嵌入维度相同，这样它们才能相加。
    # dropout: 一个介于 0 和 1 之间的浮点数，代表在训练时随机“丢弃”一部分神经元的比例，用于防止过拟合。默认为 0.1。
    # max_len: 模型能处理的句子的最大长度（比如 5000）。我们会预先计算好这么长所有位置的位置编码。
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        # super().__init__() 是一个必须的步骤，它调用了父类 nn.Module 的构造函数，以正确地初始化基类。
        super().__init__()

        # 定义一个 Dropout 层。nn.Dropout 是 PyTorch 提供的现成的层。
        # 我们将把它应用在位置编码与词嵌入相加之后的结果上。
        self.dropout = nn.Dropout(p=dropout)

        # 创建一个形状为 (max_len, d_model) 的全零张量（tensor），用来存放我们的位置编码。
        # 张量是 PyTorch 中最基本的数据结构，可以看作是多维数组。
        pe = torch.zeros(max_len, d_model)

        # 创建一个形状为 (max_len, 1) 的张量，代表句子的位置索引，即 [0, 1, 2, ..., max_len-1]。
        # torch.arange(0, max_len, dtype=torch.float) 会生成一个一维张量 [0., 1., ..., max_len-1.]。
        # .unsqueeze(1) 会在第 1 维（从0开始数）增加一个维度，将形状从 [max_len] 变为 [max_len, 1]。
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # 这是计算位置编码公式中分母的部分：10000^(2i / d_model)。
        # torch.arange(0, d_model, 2).float() 生成 [0, 2, 4, ..., d_model-2]，代表公式中的 2i。
        # 整个表达式计算出了一个包含 d_model/2 个值的张量，用于后续和 position 相乘。
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # 使用切片操作和广播机制，同时计算所有偶数维度（0, 2, 4, ...）的位置编码值。
        # pe[:, 0::2] 表示选取所有行，但只选取从第 0 列开始，步长为 2 的列（即偶数列）。
        # position * div_term 会利用广播（broadcasting）机制，(max_len, 1) 的 position 会和 (d_model/2) 的 div_term 相乘，
        # 得到一个 (max_len, d_model/2) 的结果。
        # 最后用 torch.sin 计算正弦值。
        pe[:, 0::2] = torch.sin(position * div_term)

        # 同理，计算所有奇数维度（1, 3, 5, ...）的位置编码值。
        # pe[:, 1::2] 表示选取所有行，但只选取从第 1 列开始，步长为 2 的列（即奇数列）。
        pe[:, 1::2] = torch.cos(position * div_term)

        # 【修正1】: 调整 pe 的形状以匹配 (batch_size, seq_len, d_model) 的输入格式。
        # 我们之前的代码对 pe 进行了复杂且错误的变形。
        # 正确的做法是，我们只需要在最前面增加一个维度，作为“批次”维度。
        # 这样 pe 的形状从 (max_len, d_model) 变为 (1, max_len, d_model)。
        # 这个形状可以通过广播（broadcasting）机制，轻松地与 (batch_size, seq_len, d_model) 的输入相加。
        pe = pe.unsqueeze(0)

        # self.register_buffer('pe', pe) 是一个重要的方法。
        # 它将 pe 这个张量注册为模型的“缓冲区”(buffer)。
        # 这意味着 pe 是模型状态的一部分（会和模型一起保存、加载，一起被移动到 GPU），
        # 但它不是模型的参数（parameters），所以在模型训练时，它的值不会被梯度下降更新。
        # 这正是我们想要的，因为位置编码是固定的，不需要学习。
        self.register_buffer('pe', pe)

    # forward 方法定义了当数据通过这个模块时，需要执行的计算。
    # x: 输入的张量，代表词嵌入。它的形状是 (batch_size, seq_len, d_model)。
    def forward(self, x):
        # 【修正2】: 修正切片操作，并正确地加上位置编码。
        # x.size(1) 现在正确地代表了序列长度 seq_len。
        # self.pe[:, :x.size(1), :] 的意思是：
        # - 第一个 ':' 表示取批次维度的所有数据（我们只有一个，就是那个 1）。
        # - ':x.size(1)' 表示取序列长度维度的前 seq_len 个位置编码。
        # - 第二个 ':' 表示取 d_model 维度的所有数据。
        # 最终切片出来的 pe 形状是 (1, seq_len, d_model)。
        # 当它和形状为 (batch_size, seq_len, d_model) 的 x 相加时，PyTorch 会自动将 pe 的第一维复制 batch_size 次，完成相加。
        x = x + self.pe[:, :x.size(1), :]

        # 将相加后的结果通过 dropout 层，然后返回。
        return self.dropout(x)



In [3]:
# 定义一个名为 MultiHeadAttention 的类，它同样继承自 nn.Module。
# 这是我们 Transformer 模型的核心引擎。
class MultiHeadAttention(nn.Module):
    # 类的构造函数。
    # d_model: 词嵌入的维度，它必须能被头的数量整除。
    # num_heads: 注意力“头”的数量。多头允许模型从不同角度关注信息。
    # dropout: Dropout 的比例，默认为 0.1。
    def __init__(self, d_model, num_heads, dropout=0.1):
        # 必须的步骤：调用父类 nn.Module 的构造函数。
        super().__init__()

        # 使用 assert 语句进行一个健全性检查。
        # assert 是一个断言，如果后面的条件为 False，程序就会在这里报错。
        # 这里我们确保 d_model 必须能够被 num_heads 整除。
        # 比如，如果 d_model=512, num_heads=8，那么 512 % 8 == 0，条件为 True，程序继续。
        # 如果 d_model=512, num_heads=7，条件为 False，程序会报错，提示我们参数设置有问题。
        assert d_model % num_heads == 0

        # 初始化实例变量。
        self.d_model = d_model      # 模型的总维度
        self.num_heads = num_heads  # 注意力头的数量
        self.d_k = d_model // num_heads # 每个头的维度。// 是整数除法。例如 512 // 8 = 64。

        # 定义四个线性层（Linear Layer），它们本质上就是全连接层，用来进行线性变换（矩阵乘法）。
        # nn.Linear(input_features, output_features)
        # 这里的四个线性层就对应我们理论中学到的 Wq, Wk, Wv, Wo 权重矩阵。
        self.query = nn.Linear(d_model, d_model) # 用于生成 Query 向量
        self.key = nn.Linear(d_model, d_model)   # 用于生成 Key 向量
        self.value = nn.Linear(d_model, d_model) # 用于生成 Value 向量

        # 这是最后一个线性层，对应 Wo 矩阵。它接收拼接后的多头注意力结果，并输出最终的 d_model 维度向量。
        self.fc_out = nn.Linear(d_model, d_model)

        # 定义一个 Dropout 层。
        self.dropout = nn.Dropout(dropout)

    # forward 方法定义了前向传播的逻辑。
    # query, key, value: 这三个是输入的 Q, K, V 向量。
    #   - 在 Encoder 的自注意力层中，这三个输入是完全相同的（都等于上一层的输出）。
    #   - 在 Decoder 的 Encoder-Decoder 注意力层中，Query 来自 Decoder，而 Key 和 Value 来自 Encoder 的输出。
    # mask: 掩码，用于告诉模型哪些部分是填充（padding）的，不需要关注。或者在 Decoder 中用于防止看到未来的词。
    def forward(self, query, key, value, mask=None):
        # 获取 batch_size（批次大小），也就是一次性处理多少个句子。
        # query.shape 是一个元组，例如 (32, 100, 512)，代表 (batch_size, sequence_length, d_model)。
        # query.shape[0] 就是获取第一个维度的大小，即 32。
        batch_size = query.shape[0]

        # 1. 将输入的 query, key, value 通过我们定义的线性层，进行线性变换。
        #    这步相当于乘以 Wq, Wk, Wv 矩阵。
        #    输入形状: (batch_size, seq_len, d_model)
        #    输出形状: (batch_size, seq_len, d_model)
        Q = self.query(query)
        K = self.key(key)
        V = self.value(value)

        # 2. 将变换后的 Q, K, V 进行形状重塑（reshape），以便进行多头注意力的计算。
        #    我们需要把 d_model 这个维度拆分成 num_heads 和 d_k 两个维度。
        #    .view() 函数用于改变张量的形状。
        #    原始形状: (batch_size, seq_len, d_model)
        #    目标形状: (batch_size, seq_len, num_heads, d_k)
        #    .transpose(1, 2) 用于交换维度 1 和 2。
        #    最终形状: (batch_size, num_heads, seq_len, d_k)
        #    这么做是为了让每个头都能独立地处理整个序列。
        Q = Q.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # 3. 计算注意力得分。这对应于我们理论中的公式 (Q * K^T) / sqrt(d_k)
        #    torch.matmul() 执行矩阵乘法。
        #    K.transpose(-2, -1) 将 K 的最后两个维度进行转置。
        #    K 的形状是 (batch_size, num_heads, seq_len_k, d_k)
        #    转置后 K^T 的形状是 (batch_size, num_heads, d_k, seq_len_k)
        #    Q * K^T 的结果 `energy` 的形状是 (batch_size, num_heads, seq_len_q, seq_len_k)
        energy = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        # 4. 应用掩码（mask）。
        #    如果传入了 mask，就需要将 mask 中值为 0 的位置（通常是 padding 的位置）在 energy 中对应的值设为一个非常小的负数。
        #    这样，在下一步进行 softmax 时，这些位置的概率就会趋近于 0，相当于模型忽略了它们。
        #    mask == 0 会创建一个布尔张量，padding 的位置是 True，其他位置是 False。
        #    .masked_fill_() 是一个原地操作，将 energy 中对应 True 的位置填充为 -1e9 (一个非常小的数)。
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e9"))

        # 5. 对得分进行 softmax 归一化，得到注意力权重。
        #    dim=-1 表示在最后一个维度上进行 softmax 操作，确保每一行的权重加起来等于 1。
        #    `attention` 的形状和 `energy` 相同: (batch_size, num_heads, seq_len_q, seq_len_k)
        attention = torch.softmax(energy, dim=-1)

        # 应用 dropout。
        attention = self.dropout(attention)

        # 6. 将注意力权重与 V 相乘，得到加权的 Value。
        #    torch.matmul(attention, V) 的结果 `x` 的形状是 (batch_size, num_heads, seq_len_q, d_k)
        #    这代表了每个头计算出的上下文向量。
        x = torch.matmul(attention, V)

        # 7. 拼接多头的结果。
        #    我们需要把多头计算的结果重新组合成一个 d_model 维度的向量。
        #    .transpose(1, 2) 将形状变回 (batch_size, seq_len_q, num_heads, d_k)。
        #    .contiguous() 是一个 PyTorch 的操作，它确保张量在内存中是连续存储的，这是 .view() 操作所必需的。
        #    .view() 将最后两个维度 (num_heads, d_k) 重新合并为 d_model。
        #    最终形状: (batch_size, seq_len_q, d_model)
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        # 8. 将拼接后的结果通过最后一个线性层（对应 Wo 矩阵），得到最终的输出。
        #    输入形状: (batch_size, seq_len_q, d_model)
        #    输出形状: (batch_size, seq_len_q, d_model)
        x = self.fc_out(x)

        # 返回最终的输出张量。
        return x


In [4]:
# 定义一个名为 PositionwiseFeedforward 的类，它继承自 nn.Module。
# "Positionwise" 意味着这个网络会独立地、相同地应用于输入序列中的每一个位置（每一个词）。
class PositionwiseFeedforward(nn.Module):
    # 类的构造函数。
    # d_model: 输入和输出的维度。
    # d_ff: 内部隐藏层的维度。在原版 Transformer 论文中，这个值通常是 d_model 的 4 倍（例如, d_model=512, d_ff=2048）。
    # dropout: Dropout 的比例，默认为 0.1。
    def __init__(self, d_model, d_ff, dropout=0.1):
        # 调用父类 nn.Module 的构造函数。
        super().__init__()

        # 定义第一个线性层。它将输入从 d_model 维度扩展到 d_ff 维度。
        self.fc1 = nn.Linear(d_model, d_ff)
        # 定义第二个线性层。它将维度从 d_ff 压缩回 d_model。
        self.fc2 = nn.Linear(d_ff, d_model)
        # 定义 Dropout 层。
        self.dropout = nn.Dropout(dropout)
        # 定义 ReLU 激活函数。它为模型引入了非线性，使得模型能学习更复杂的关系。
        self.relu = nn.ReLU()

    # forward 方法定义了前向传播的逻辑。
    # x: 输入张量，形状为 (batch_size, seq_len, d_model)。
    def forward(self, x):
        # 1. 将输入 x 通过第一个线性层 (fc1)。
        #    形状变化: (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_ff)
        x = self.fc1(x)

        # 2. 将结果通过 ReLU 激活函数。
        #    形状不变: (batch_size, seq_len, d_ff)
        x = self.relu(x)

        # 3. 将结果通过 Dropout 层。
        #    形状不变: (batch_size, seq_len, d_ff)
        x = self.dropout(x)

        # 4. 将结果通过第二个线性层 (fc2)。
        #    形状变化: (batch_size, seq_len, d_ff) -> (batch_size, seq_len, d_model)
        x = self.fc2(x)

        # 返回最终的输出张量。
        return x


In [5]:
# 定义一个名为 EncoderLayer 的类，它继承自 nn.Module。
# 这是构成整个 Encoder 的基本单元。
class EncoderLayer(nn.Module):
    # 类的构造函数。
    # d_model: 模型的维度。
    # num_heads: 多头注意力的头数。
    # d_ff: 前馈网络内部的维度。
    # dropout: Dropout 的比例。
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        # 调用父类构造函数。
        super().__init__()

        # 实例化一个多头自注意力模块。我们直接使用上面定义的 MultiHeadAttention 类。
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)

        # 实例化一个前馈神经网络模块。我们直接使用上面定义的 PositionwiseFeedforward 类。
        self.feed_forward = PositionwiseFeedforward(d_model, d_ff, dropout)

        # 定义两个层归一化（Layer Normalization）模块。
        # nn.LayerNorm 会对输入的最后一个维度（这里是 d_model）进行归一化。
        # 这有助于稳定训练过程，加速收敛。
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        # 定义两个 Dropout 层。
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    # forward 方法定义了前向传播的逻辑。
    # x: 输入张量，形状为 (batch_size, seq_len, d_model)。
    # mask: 掩码，用于在自注意力计算中忽略 padding 的部分。
    def forward(self, x, mask):
        # 1. --- 第一个子层：多头自注意力 ---

        #    a. 计算多头自注意力的输出。
        #       注意，对于自注意力，query, key, value 都是相同的，都等于输入 x。
        attn_output = self.self_attn(x, x, x, mask)

        #    b. 残差连接 (Add) 和层归一化 (Norm)。
        #       首先，将注意力层的输出通过一个 dropout 层。
        #       然后，将 dropout 后的结果与原始输入 x 相加（这就是残差连接）。
        #       最后，将相加的结果通过第一个层归一化模块 (norm1)。
        #       这个 `x + ...` 的操作就是 Add & Norm 中的 "Add"。
        #       `self.norm1(...)` 就是 "Norm"。
        x = self.norm1(x + self.dropout1(attn_output))

        # 2. --- 第二个子层：前馈神经网络 ---

        #    a. 计算前馈神经网络的输出。
        #       输入是上一个子层归一化后的结果 x。
        forward_output = self.feed_forward(x)

        #    b. 再次进行残差连接 (Add) 和层归一化 (Norm)。
        #       首先，将前馈网络的输出通过第二个 dropout 层。
        #       然后，将 dropout 后的结果与该子层的输入 x 相加。
        #       最后，将相加的结果通过第二个层归一化模块 (norm2)。
        x = self.norm2(x + self.dropout2(forward_output))

        # 返回编码器层的最终输出。
        # 输出的形状与输入相同: (batch_size, seq_len, d_model)。
        return x


In [6]:
# 定义一个名为 DecoderLayer 的类，它继承自 nn.Module。
# 这是构成整个 Decoder 的基本单元。
class DecoderLayer(nn.Module):
    # 类的构造函数。
    # d_model: 模型的维度。
    # num_heads: 多头注意力的头数。
    # d_ff: 前馈网络内部的维度。
    # dropout: Dropout 的比例。
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        # 调用父类构造函数。
        super().__init__()

        # 实例化第一个多头注意力模块，用于解码器自身的“带掩码自注意力”。
        # 我们仍然使用 MultiHeadAttention 类，之后在前向传播时传入一个特殊的掩码即可。
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)

        # 实例化第二个多头注意力模块，用于“编码器-解码器注意力”。
        # 它关注编码器的输出。
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads, dropout)

        # 实例化一个前馈神经网络模块。
        self.feed_forward = PositionwiseFeedforward(d_model, d_ff, dropout)

        # 定义三个层归一化模块，因为我们有三个子层。
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)

        # 定义三个 Dropout 层。
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    # forward 方法定义了前向传播的逻辑。
    # x: 解码器的输入张量，形状为 (batch_size, target_seq_len, d_model)。
    # enc_output: 编码器的输出张量，形状为 (batch_size, source_seq_len, d_model)。这是解码器需要关注的上下文。
    # target_mask: 目标语言的掩码。这既包含了对 padding 的掩码，也包含了防止看到未来的“顺序掩码”(look-ahead mask)。
    # source_mask: 源语言的掩码。这只包含了对 padding 的掩码。
    def forward(self, x, enc_output, source_mask, target_mask):
        # 1. --- 第一个子层：带掩码的多头自注意力 ---

        #    a. 计算多头自注意力的输出。
        #       这里的 query, key, value 都是解码器的输入 x。
        #       我们传入 target_mask 来确保每个位置只能关注到它自己和它前面的位置。
        self_attn_output = self.self_attn(x, x, x, target_mask)

        #    b. 残差连接和层归一化。
        x = self.norm1(x + self.dropout1(self_attn_output))

        # 2. --- 第二个子层：编码器-解码器注意力 ---

        #    a. 计算编码器-解码器注意力的输出。
        #       这里的关键是：
        #       - Query (Q) 来自于上一个子层的输出 x (解码器自身的信息)。
        #       - Key (K) 和 Value (V) 都来自于编码器的输出 enc_output (源语言句子的信息)。
        #       - 我们传入 source_mask，因为它作用于 K 和 V，需要屏蔽掉源语言句子中的 padding 部分。
        enc_dec_attn_output = self.enc_dec_attn(x, enc_output, enc_output, source_mask)

        #    b. 残差连接和层归一化。
        x = self.norm2(x + self.dropout2(enc_dec_attn_output))

        # 3. --- 第三个子层：前馈神经网络 ---

        #    a. 计算前馈神经网络的输出。
        forward_output = self.feed_forward(x)

        #    b. 残差连接和层归一化。
        x = self.norm3(x + self.dropout3(forward_output))

        # 返回解码器层的最终输出。
        # 输出的形状与输入 x 相同: (batch_size, target_seq_len, d_model)。
        return x


In [7]:
# 定义一个名为 Encoder 的类，它继承自 nn.Module。
# 整个编码器部分由 N 个相同的 EncoderLayer 堆叠而成。
class Encoder(nn.Module):
    # 类的构造函数。
    # input_dim: 输入词典的大小（比如源语言有 30000 个不同的词）。
    # d_model: 模型的维度。
    # num_layers: 编码器层 (EncoderLayer) 的数量 (原论文中是 6)。
    # num_heads: 多头注意力的头数。
    # d_ff: 前馈网络内部的维度。
    # dropout: Dropout 的比例。
    def __init__(self, input_dim, d_model, num_layers, num_heads, d_ff, dropout=0.1):
        # 调用父类构造函数。
        super().__init__()

        # 定义词嵌入层 (Embedding Layer)。
        # nn.Embedding 会创建一个查找表（lookup table），将每个词的索引（一个整数）映射到一个 d_model 维的向量。
        self.embedding = nn.Embedding(input_dim, d_model)

        # 实例化我们之前创建的位置编码模块。
        self.pos_encoding = PositionalEncoding(d_model, dropout)

        # 创建一个模块列表 (ModuleList) 来存放所有的编码器层。
        # nn.ModuleList 是一个特殊的列表，它可以正确地注册它包含的所有模块，让 PyTorch 知道它们是模型的一部分。
        # 我们使用一个 for 循环来创建 num_layers 个 EncoderLayer 实例。
        self.layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        # 定义一个 Dropout 层。
        self.dropout = nn.Dropout(dropout)

    # forward 方法定义了前向传播的逻辑。
    # src: 源语言句子的输入张量，形状为 (batch_size, src_len)，内容是词的索引。
    # mask: 源语言的掩码。
    def forward(self, src, mask):
        # 1. 将输入的词索引通过嵌入层，转换为词向量。
        #    形状变化: (batch_size, src_len) -> (batch_size, src_len, d_model)
        src = self.embedding(src)

        # 2. 将位置编码添加到词向量上。
        #    形状不变: (batch_size, src_len, d_model)
        src = self.pos_encoding(src)

        # 3. 让数据依次通过 ModuleList 中的每一个编码器层。
        #    我们使用一个 for 循环来遍历 self.layers。
        #    在每一层，输入是上一层的输出。
        for layer in self.layers:
            src = layer(src, mask)

        # 返回编码器最终的输出。
        # 形状: (batch_size, src_len, d_model)
        return src


In [8]:
# 定义一个名为 Decoder 的类，它继承自 nn.Module。
# 整个解码器部分由 N 个相同的 DecoderLayer 堆叠而成。
class Decoder(nn.Module):
    # 类的构造函数。
    # output_dim: 输出词典的大小（比如目标语言有 32000 个不同的词）。
    # d_model, num_layers, num_heads, d_ff, dropout: 参数含义与 Encoder 相同。
    def __init__(self, output_dim, d_model, num_layers, num_heads, d_ff, dropout=0.1):
        # 调用父类构造函数。
        super().__init__()

        # 定义目标语言的词嵌入层。
        self.embedding = nn.Embedding(output_dim, d_model)

        # 实例化位置编码模块。
        self.pos_encoding = PositionalEncoding(d_model, dropout)

        # 创建一个 ModuleList 来存放所有的解码器层。
        self.layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        # 定义一个最终的线性层。
        # 这个层的作用是将解码器最后一层的输出（d_model 维度）映射到整个目标词典的大小（output_dim 维度）。
        # 这样，对于每个位置，我们都能得到一个代表每个词得分的向量。
        self.fc_out = nn.Linear(d_model, output_dim)

        # 定义 Dropout 层。
        self.dropout = nn.Dropout(dropout)

    # forward 方法定义了前向传播的逻辑。
    # trg: 目标语言句子的输入张量，形状为 (batch_size, trg_len)。
    # enc_src: 编码器的输出，形状为 (batch_size, src_len, d_model)。
    # trg_mask: 目标语言的掩码。
    # src_mask: 源语言的掩码。
    def forward(self, trg, enc_src, trg_mask, src_mask):
        # 1. 将目标语言的词索引通过嵌入层，转换为词向量。
        #    形状变化: (batch_size, trg_len) -> (batch_size, trg_len, d_model)
        trg = self.embedding(trg)

        # 2. 将位置编码添加到词向量上。
        #    形状不变: (batch_size, trg_len, d_model)
        trg = self.pos_encoding(trg)

        # 3. 让数据依次通过 ModuleList 中的每一个解码器层。
        #    每一层都需要接收上一步的输出 trg, 编码器的输出 enc_src, 以及两种掩码。
        for layer in self.layers:
            trg = layer(trg, enc_src, src_mask, trg_mask)

        # 4. 将解码器最后一层的输出通过最终的线性层 fc_out。
        #    形状变化: (batch_size, trg_len, d_model) -> (batch_size, trg_len, output_dim)
        output = self.fc_out(trg)

        # 返回最终的输出（也称为 logits）。
        return output


In [9]:
# 定义最终的 Transformer 模型类。
class Transformer(nn.Module):
    # 类的构造函数。
    # encoder: 一个 Encoder 类的实例。
    # decoder: 一个 Decoder 类的实例。
    # src_pad_idx: 源语言中填充符号 <pad> 的索引。
    # trg_pad_idx: 目标语言中填充符号 <pad> 的索引。
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx):
        # 调用父类构造函数。
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx

    # 定义一个方法来创建源语言的掩码。
    # src: 源语言输入张量，形状为 (batch_size, src_len)。
    def make_src_mask(self, src):
        # 1. 检查 src 张量中哪些元素等于填充索引。
        #    (src != self.src_pad_idx) 会生成一个布尔张量，padding 的位置是 False，非 padding 是 True。
        #    形状: (batch_size, src_len)
        # 2. 在最后两个维度上增加一个维度，以匹配多头注意力的期望形状。
        #    .unsqueeze(1).unsqueeze(2) 会将形状变为 (batch_size, 1, 1, src_len)。
        #    这个形状可以和注意力得分矩阵 (batch_size, num_heads, seq_len, seq_len) 进行广播。
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask

    # 定义一个方法来创建目标语言的掩码。
    # trg: 目标语言输入张量，形状为 (batch_size, trg_len)。
    def make_trg_mask(self, trg):
        # 1. 创建 padding 掩码，逻辑与 src_mask 相同。
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        # 形状: (batch_size, 1, 1, trg_len)

        # 2. 创建“顺序”掩码 (look-ahead mask)，防止看到未来的词。
        trg_len = trg.shape[1]
        #    torch.tril() 会创建一个下三角矩阵。
        #    torch.ones((trg_len, trg_len)) 创建一个全 1 的方阵。
        #    torch.tril(...) 会将这个方阵的右上部分变为 0。
        #    例如 trg_len=3, 结果是:
        #    [[1, 0, 0],
        #     [1, 1, 0],
        #     [1, 1, 1]]
        #    .to(device) 确保掩码和数据在同一个设备上（CPU 或 GPU）。
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device=device)).bool()
        # 形状: (trg_len, trg_len)

        # 3. 将 padding 掩码和顺序掩码结合起来。
        #    使用逻辑与 (&) 操作。只有当两个掩码在某个位置都为 True 时，最终结果才为 True。
        #    trg_pad_mask 的形状是 (batch_size, 1, 1, trg_len)
        #    trg_sub_mask 的形状是 (trg_len, trg_len)
        #    PyTorch 的广播机制会自动处理这两个不同形状的张量。
        trg_mask = trg_pad_mask & trg_sub_mask

        return trg_mask

    # forward 方法定义了整个模型的前向传播。
    # src: 源语言输入。
    # trg: 目标语言输入。
    def forward(self, src, trg):
        # 1. 创建源语言和目标语言的掩码。
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        # 2. 将源语言和掩码输入编码器，得到编码器的输出。
        enc_src = self.encoder(src, src_mask)

        # 3. 将目标语言、编码器输出以及两种掩码输入解码器，得到最终的输出。
        output = self.decoder(trg, enc_src, trg_mask, src_mask)

        # 返回输出。
        return output


In [10]:
# --- 模型超参数定义 ---

# INPUT_DIM: 输入词典的大小。
# 假设我们的源语言（例如中文）词典中有 5000 个独特的词。
INPUT_DIM = 5000

# OUTPUT_DIM: 输出词典的大小。
# 假设我们的目标语言（例如英文）词典中有 5000 个独特的词。
OUTPUT_DIM = 5000

# D_MODEL: 模型的维度，也就是词嵌入向量的维度。
# 这是 Transformer 模型中的一个核心维度，它贯穿整个模型。
# 论文中设置为 512。
D_MODEL = 512

# NUM_LAYERS: 编码器和解码器中堆叠的层数。
# 论文中设置为 6。
NUM_LAYERS = 6

# NUM_HEADS: 多头注意力机制中的“头”数。
# 注意：D_MODEL 必须能够被 NUM_HEADS 整除 (512 % 8 == 0)。
# 论文中设置为 8。
NUM_HEADS = 8

# D_FF: 前馈神经网络内部的隐藏层维度。
# 论文中建议设置为 D_MODEL 的 4 倍。
D_FF = 2048

# DROPOUT: Dropout 的比例，用于防止过拟合。
# 论文中设置为 0.1。
DROPOUT = 0.1

# SRC_PAD_IDX: 源语言中，用于填充（padding）的特殊符号 <pad> 在词典中的索引。
# 我们假设它的索引是 0。这个值在创建掩码时至关重要。
SRC_PAD_IDX = 0

# TRG_PAD_IDX: 目标语言中，用于填充的特殊符号 <pad> 在词典中的索引。
# 我们也假设它的索引是 0。
TRG_PAD_IDX = 0


In [11]:
# --- 模型实例化 ---

# 1. 实例化编码器 (Encoder)。
#    我们将上面定义的所有相关超参数作为参数传入。
enc = Encoder(INPUT_DIM,
              D_MODEL,
              NUM_LAYERS,
              NUM_HEADS,
              D_FF,
              DROPOUT)

# 2. 实例化解码器 (Decoder)。
dec = Decoder(OUTPUT_DIM,
              D_MODEL,
              NUM_LAYERS,
              NUM_HEADS,
              D_FF,
              DROPOUT)

# 3. 实例化最终的 Transformer 模型。
#    它接收我们刚刚创建的编码器和解码器实例，以及填充索引。
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX)

# 4. 将模型移动到我们之前设置好的设备上（GPU 或 CPU）。
#    .to(device) 是一个 PyTorch 方法，用于将模型的所有参数和缓冲区移动到指定的设备。
#    这是使用 GPU 加速所必需的步骤。
model.to(device)

# (可选) 打印一下模型的总参数数量，感受一下它的规模。
def count_parameters(model):
    # sum(p.numel() for p in model.parameters() if p.requires_grad) 是一个 Python 的生成器表达式。
    # model.parameters() 会返回模型所有可学习的参数（权重和偏置）。
    # p.requires_grad 检查这个参数是否需要计算梯度（即是否在训练中被更新）。
    # p.numel() 返回参数 p 中元素的总数。
    # sum(...) 将所有参数的元素数量加起来，得到模型的总参数量。
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# 打印总参数量，并用逗号进行格式化，使其更易读。
print(f'这个模型共有 {count_parameters(model):,} 个可训练的参数')


这个模型共有 51,823,496 个可训练的参数


In [12]:
# --- 健全性检查 ---

# 设置一些用于测试的维度
BATCH_SIZE = 128  # 批次大小，即一次处理 128 个句子。
SRC_LEN = 30      # 源语言句子的长度，假设为 30 个词。
TRG_LEN = 35      # 目标语言句子的长度，假设为 35 个词。

# 创建一个假的源语言输入张量。
# torch.randint(low, high, size) 会生成一个在 [low, high) 区间内的随机整数张量。
# 我们生成的值在 [0, INPUT_DIM) 之间，模拟词典中的词索引。
# size=(BATCH_SIZE, SRC_LEN) 指定了张量的形状。
# .to(device) 确保这个数据张量也在 GPU 上，与模型在同一个设备。
src = torch.randint(0, INPUT_DIM, (BATCH_SIZE, SRC_LEN)).to(device)

# 创建一个假的目标语言输入张量。
trg = torch.randint(0, OUTPUT_DIM, (BATCH_SIZE, TRG_LEN)).to(device)

# 打印一下输入数据的形状，以便对比。
print("输入 src 的形状:", src.shape)
print("输入 trg 的形状:", trg.shape)
print("-" * 30) # 打印一条分割线

# 核心步骤：将假数据喂给模型，进行一次完整的前向传播。
# model(src, trg) 会自动调用我们 Transformer 类中定义的 forward 方法。
output = model(src, trg)

# 打印输出数据的形状。
print("模型输出 output 的形状:", output.shape)
print("-" * 30)

# --- 验证输出形状 ---
# 我们期望的输出形状应该是 (BATCH_SIZE, TRG_LEN, OUTPUT_DIM)
# 因为对于目标序列中的每一个词，模型都应该预测一个在整个目标词典上的得分分布。
expected_shape = (BATCH_SIZE, TRG_LEN, OUTPUT_DIM)
if output.shape == expected_shape:
    print("🎉 恭喜！模型结构正确，健全性检查通过！")
    print(f"输出形状 ({output.shape}) 与期望形状 ({expected_shape}) 完全一致。")
else:
    print("😥 模型结构似乎有问题，请检查代码。")
    print(f"输出形状是 {output.shape}, 但我们期望的是 {expected_shape}。")



输入 src 的形状: torch.Size([128, 30])
输入 trg 的形状: torch.Size([128, 35])
------------------------------
模型输出 output 的形状: torch.Size([128, 35, 5000])
------------------------------
🎉 恭喜！模型结构正确，健全性检查通过！
输出形状 (torch.Size([128, 35, 5000])) 与期望形状 ((128, 35, 5000)) 完全一致。
